In [1]:
! pip install pyautogen

In [2]:
import os

import autogen
import json
import pandas as pd

import time  
start_time = time.time()  

import logging
logging.basicConfig(level=logging.ERROR)

# Put your api key in the environment variable OPENAI_API_KEY
llm_config = {
    "model": "claude-3-5-sonnet-20241022",
    "api_key": "",
    "base_url": "https://api.anthropic.com/",
}

In [4]:
# Create Agents, GroupChat, and GroupChatManager in line with the original group chat
gpt4_config = llm_config

# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  
    max_consecutive_auto_reply=3,  
    code_execution_config=False,
)

# Planner Agent
planner = autogen.AssistantAgent(
    name="Planner",
    llm_config=llm_config,
    system_message="""
You are a task router. Based on the user's instruction, route the task as follows:

1. If the instruction contains "Extract salary information from the job ad", respond with SALARY. This task should be handled by the Salary Agent.
2. If the instruction contains "Extract seniority level from the job ad", respond with SENIORITY. This task should be handled by the Seniority Agent.
3. If the instruction contains "Extract work arrangement from the job ad", respond with ARRANGEMENT. This task should be handled by the Arrangement Agent.

Do not ask the user for clarification. Just respond with one keyword: SALARY, SENIORITY, or ARRANGEMENT.

If you receive 'TERMINATE' or the task is completed, end the conversation immediately.
"""
)

# Salary Agent
salary_agent = autogen.AssistantAgent(
    name="Salary",
    llm_config=llm_config,
    system_message="""You specialize in extracting salary information from job advertisements. Your output must follow the format below:
    [minimum number]-[maximum number]-[currency]-[time unit]
    For example: 32-45-AUD-HOURLY or 80000-100000-USD-YEARLY
    If the information is incomplete or missing, output: 0-0-None-None
    Only extract salary information. Do not respond to any other questions."""
)

# Seniority Agent
seniority_agent = autogen.AssistantAgent(
    name="Seniority",
    llm_config=llm_config,
    system_message="""You specialize in extracting seniority level information from job advertisements.
    You must choose the most appropriate seniority level from the following list:
    experienced, intermediate, senior, entry level, assistant, lead, head, junior, graduate, trainee, associate, principal, apprentice, executive, manager, director, entry-level, chief, deputy, mid-level, specialist, experienced assistant, supervisor, qualified, student, board, graduate/junior, senior associate, mid-senior
    Only extract the seniority level. Do not respond to any other questions."""
)

# Arrangement Agent
arrangement_agent = autogen.AssistantAgent(
    name="Arrangement",
    llm_config=llm_config,
    system_message="""You specialize in extracting work arrangement information from job advertisements. Your output must be one of the following options:
    OnSite, Remote, Hybrid
    If the work arrangement cannot be determined, return UNKNOWN.
    Do not output full sentences or explanations. The output must be a single word only!
    Only extract the work arrangement. Do not respond to any other questions."""
)

# Critic Agent
critic_agent = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="""You are responsible for verifying whether the outputs from other agents meet the required format:

    1. Salary output must be in the format: number-number-currency-time_unit, or 0-0-None-None (if salary information is not available).
    2. Seniority output must be one of the following valid levels:
    UNKNOWN, experienced, intermediate, senior, entry level, assistant, lead, head, junior, graduate, trainee, associate, principal, apprentice, executive, manager, director, entry-level, chief, deputy, mid-level, specialist, experienced assistant, supervisor, qualified, student, board, graduate/junior, senior associate, mid-senior
    3. Arrangement output must be one of: OnSite, Hybrid, Remote, or UNKNOWN.

    You do not need to extract information from the text — only check whether the output from the agents conforms to these formats.

    If the output is valid, respond with 'TERMINATE'.

    If the output is invalid, use `@AgentName` to explicitly request that agent to retry. 
    Only one agent should respond after your message.
    Do not ask the planner or the user to take any action.
    Terminate the process if the same agent fails 3 times."""
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, planner, salary_agent, seniority_agent, arrangement_agent, critic_agent],
    messages=[],  
    max_round=10,  
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

def print_groupchat_log(groupchat):
    print("\n📜 GroupChat Log:")
    for idx, msg in enumerate(groupchat.messages):
        name = msg.get("name", "Unknown")
        content = msg.get("content", "").strip()
        print(f"🔹 [{idx+1}] {name}: {content[:200]}{'...' if len(content) > 200 else ''}")

def process_prompt(prompt_text):
    # 路由
    plan_reply = planner.initiate_chat(recipient=manager, message=prompt_text)
    route = plan_reply.summary.strip().upper()

    print_groupchat_log(groupchat)  

    if route == "SALARY":
        worker = salary_agent
    elif route == "SENIORITY":
        worker = seniority_agent
    elif route == "ARRANGEMENT":
        worker = arrangement_agent
    else:
        return None

    worker_reply = worker.initiate_chat(recipient=manager, message=prompt_text)
    chat_content = worker_reply.chat_history[-1]["content"].strip()
    extraction = chat_content

    print_groupchat_log(groupchat)  

    critic_reply = critic_agent.initiate_chat(recipient=manager, message=worker_reply.chat_history[-1]["content"])

    print_groupchat_log(groupchat)  

    return {
        "route": route,
        "extracted": extraction,
        "critic": critic_reply.summary.strip()
    }

def get_seniority_output(prompt_text):
    reply = seniority_agent.initiate_chat(recipient=manager, message=prompt_text)
    
    for msg in reversed(reply.chat_history):
        if msg.get("name") == "Seniority":
            seniority_output = msg["content"].strip()
            print(f"✅ Extracted seniority: {seniority_output}")  
            return seniority_output
    
    return "ERROR: No output from Seniority agent"

with open('seniority_test_combined.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print("✅ y_true values from JSON (Expected):")
for i in range(5):  
    entry = data[i]
    expected = entry['complete']
    print(f"Item {i+1} - y_true: {expected}")

results = []
for i, entry in enumerate(data):
    entry = data[i]
    prompt = entry['prompt']
    expected = entry['complete']
    
    print(f"\n🔍 Processing item {i+1}:")
    
    try:
        start_item_time = time.time()  
        
        predicted = get_seniority_output(prompt)

        end_item_time = time.time()  
        elapsed_item = end_item_time - start_item_time
        
        print("✅ Extracted seniority:", predicted)
        print(f"⏱️ Time used: {elapsed_item:.2f} 秒")
        
    except Exception as e:
        print("❌ Error:", e)
        predicted = "ERROR"
        elapsed_item = None  
    
    results.append({
        "index": i,
        "y_true": expected,
        "y_predicted": predicted,
        "time_seconds": round(elapsed_item, 2) if elapsed_item is not None else "ERROR"
    })

df = pd.DataFrame(results)
print("\n📋 Results Table:")
print(df)

df.to_csv("seniority_predictions.csv", index=False)

end_time = time.time()
elapsed = end_time - start_time
print(f"\n⏱️ Total time consumption：{elapsed:.2f} seconds ({elapsed/60:.2f} minutes)")

✅ y_true values from JSON (Expected):
Item 1 - y_true: senior
Item 2 - y_true: experienced
Item 3 - y_true: entry level
Item 4 - y_true: senior
Item 5 - y_true: intermediate

🔍 Processing item 1:
Seniority (to chat_manager):

Extract seniority level from the job ad. 

job ad: 
Financial Planning and Modelling Manager . Help us create better connected futures.
Make your mark. See the difference. Be recognised.
Solve high-impact, complex challenges
Work with aligned engaged team members
About us
At BAI, we draw from our rich history to create even better connected futures.
We are both big enough to deliver and maintain large-scale operations and nimble enough to make things happen.
We deliver Wi-Fi and cellular connectivity on subways from New York to Hong Kong to Toronto and we keep Australians, from the cities to the outback, connected to what matters to them.
We build technology, and teams, that people want to be part of, and we have the courage to not only imagine, but to do, what re

KeyboardInterrupt: 